## Application of BPR on Movielens

In [1]:
#%load_ext watermark
%load_ext autoreload 
%autoreload 2

In [2]:
import os
import sys
import itertools
import numpy as np
import pandas as pd
from subprocess import call
from pybpr import *
import matplotlib.pyplot as plt
from functools import partial

Number of users: 8
Number of items: 10
Number of users: 4
Number of items: 5


In [3]:
%%time
#df = load_movielens_data('ml-1m')
df = load_movielens_data('ml-100k')
df.head()

CPU times: user 20.5 ms, sys: 7.5 ms, total: 28 ms
Wall time: 29.8 ms


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [87]:
df['user_id'] = df['user_id'].astype('category')
df['item_id'] = df['item_id'].astype('category')
df['user_id'] = df['user_id'].cat.codes
df['item_id'] = df['item_id'].cat.codes

In [88]:
df.user_id.unique()

,user_id,item_id,rating,timestamp,users,items
0,195,241,3,881250949,195,241
1,185,301,3,891717742,185,301
2,21,376,1,878887116,21,376
3,243,50,2,880606923,243,50
4,165,345,1,886397596,165,345
...,...,...,...,...,...,...
99995,879,475,3,880175444,879,475
99996,715,203,5,879795543,715,203
99997,275,1089,1,874795795,275,1089
99998,12,224,2,882399156,12,224


In [90]:
dfshort_pos = df[df['rating'] > 3]
dfshort_neg = df[df['rating'] <= 3]

In [91]:
unique1 = dfshort_pos.user_id.unique()
idf = dfshort_pos.groupby('user_id')['item_id'].count()
unique2 = idf[idf>0].index.values
users1 = np.intersect1d(unique1, unique2, assume_unique=True)

unique1 = dfshort_neg.user_id.unique()
idf = dfshort_neg.groupby('user_id')['item_id'].count()
unique2 = idf[idf>0].index.values
users2 = np.intersect1d(unique1, unique2, assume_unique=True)

users = np.intersect1d(users1, users2, assume_unique=True)


unique1 = dfshort_pos.item_id.unique()
idf = dfshort_pos.groupby('item_id')['user_id'].count()
unique2 = idf[idf>0].index.values
items1 = np.intersect1d(unique1, unique2, assume_unique=True)

unique1 = dfshort_neg.user_id.unique()
idf = dfshort_neg.groupby('item_id')['user_id'].count()
unique2 = idf[idf>0].index.values
items2 = np.intersect1d(unique1, unique2, assume_unique=True)

items = np.intersect1d(items1, items2, assume_unique=True)


print(len(users), len(items))

940 913


In [92]:
ibool = (dfshort_pos.user_id.isin(users)) & (dfshort_pos.item_id.isin(items))
dfshort = dfshort_pos[ibool]
cf_pos = UserItemInteractions(
    name='MovieLens-100k',
    users=dfshort['user_id'],
    items=dfshort['item_id'],
    min_num_rating_per_user=0,
    min_num_rating_per_item=0,
    num_cores = 1
)
cf_pos.generate_train_test(user_test_ratio=0.0)
ibool = (dfshort_neg.user_id.isin(users)) & (dfshort_neg.item_id.isin(items))
dfshort = dfshort_neg[ibool]
cf_neg = UserItemInteractions(
    name='MovieLens-100k',
    users=dfshort['user_id'],
    items=dfshort['item_id'],
    min_num_rating_per_user=0,
    min_num_rating_per_item=0,
    num_cores = 1
)
cf_neg.generate_train_test(user_test_ratio=0.0)

Number of users: 940
Number of items: 912
Number of users: 940
Number of items: 913


In [85]:
bpr = BPR(
    num_features=10,
    reg_lambda=0.,
    num_iters=10,
    learning_rate = 0.5,
    batch_size=5,
    initial_std=0.01,
    seed=None
)

In [86]:
bpr.fit(cf_pos.R_train, None)

BPR:   0%|                                                         | 0/10 [00:00<?, ?it/s]

0-0: 934, 126, 700
0-0: 203, 314, 57
0-0: 140, 285, 284
0-0: 274, 293, 145
0-0: 648, 49, 942
0-1: 34, 332, 561
0-1: 922, 830, 662
0-1: 176, 185, 868
0-1: 369, 513, 116
0-1: 866, 180, 499
0-2: 532, 19, 435
0-2: 340, 875, 22
0-2: 653, 422, 616
0-2: 861, 822, 213
0-2: 79, 44, 434
0-3: 400, 650, 12
0-3: 238, 49, 702
0-3: 910, 152, 226
0-3: 132, 268, 44
0-3: 100, 180, 54
0-4: 455, 602, 547
0-4: 619, 0, 185
0-4: 283, 257, 179
0-4: 858, 14, 113
0-4: 295, 221, 66
0-5: 440, 0, 501
0-5: 872, 291, 467
0-5: 230, 180, 527
0-5: 818, 245, 157
0-5: 241, 305, 718
0-6: 148, 285, 791
0-6: 153, 285, 262
0-6: 596, 282, 21


ValueError: 'a' cannot be empty unless no samples are taken

In [ ]:

ndcg_df = pd.DataFrame(bpr.ndcg_metric)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,3))
ax[0].plot(ndcg_df['test'])
ax[1].plot(ndcg_df['train'])

In [ ]:
#bpr_ndcg_func(test=True), bpr_ndcg_func(test=False)

In [ ]:
ndcg_df